In [1]:
import pandas as pd
import numpy as np

In [4]:
# Datensaetze einlesen
la = pd.read_csv('la_2019.csv', parse_dates=["start_time","end_time"])
weather = pd.read_csv('weather_hourly_la.csv')

In [10]:
# Ueberblick ueber die Daten la
print(la.head())
print(la.count())
print(la.info())

           start_time            end_time  start_station_id  end_station_id  \
0 2019-01-01 00:07:00 2019-01-01 00:14:00              3046            3051   
1 2019-01-01 00:08:00 2019-01-01 00:14:00              3046            3051   
2 2019-01-01 00:18:00 2019-01-01 00:50:00              3030            3075   
3 2019-01-01 00:20:00 2019-01-01 00:50:00              3030            3075   
4 2019-01-01 00:22:00 2019-01-01 00:50:00              3030            3075   

  bike_id user_type start_station_name end_station_name  
0   06468   Walk-up         2nd & Hill   7th & Broadway  
1   12311   Walk-up         2nd & Hill   7th & Broadway  
2   05992   Walk-up         Main & 1st   Broadway & 9th  
3   05860   Walk-up         Main & 1st   Broadway & 9th  
4   06006   Walk-up         Main & 1st   Broadway & 9th  
start_time            290342
end_time              290342
start_station_id      290342
end_station_id        290342
bike_id               290342
user_type             290342
sta

In [9]:
# Ueberblick ueber die Daten weather
print(weather.head())
print(weather.count())
print(weather.info())

             date_time  max_temp  min_temp  precip
0  2015-01-02 01:00:00      11.7      11.7     0.0
1  2015-01-02 02:00:00      11.1      11.1     0.0
2  2015-01-02 03:00:00      11.1      11.1     0.0
3  2015-01-02 04:00:00      10.0      10.0     0.0
4  2015-01-02 05:00:00      10.0      10.0     0.0
date_time    43756
max_temp     43756
min_temp     43756
precip       43758
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43848 entries, 0 to 43847
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date_time  43756 non-null  object 
 1   max_temp   43756 non-null  float64
 2   min_temp   43756 non-null  float64
 3   precip     43758 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.3+ MB
None
